# Непрерывные счётчики времени

## TL;DR

Best practice: если требуется монотонный счётчик с высокой точностью, можно использовать `default_timer` из модуля `timeit`.

## Определения времён

*Wall time* - разница между временами конца и начала выполнения программы.

*CPU time (process time)* - время, в течение которого CPU выполнял программу.

*User time* - время, в течение которого CPU выполнял программу в user space.

*System time* - время, в течение которого CPU выполнял программу в kernel space.

## `time`

In [1]:
import time

time.sleep(1)

Задерживает исполнение вызывающего треда на переданное число секунд. Если получен сигнал и обработчик сигнала бросает исключение задержка может длится меньше указанного времени.

In [2]:
import warnings
warnings.filterwarnings('ignore') # deprecation warnings 

time_0 = time.clock()
time.sleep(1)
time_1 = time.clock()
time_1 - time_0

0.0036609999999999143

Unix : возвращает *current processor time* в секундах с момента начала процесса или c момента первого вызова `clock()`. Поведение и точность зависят от платформы.

Windows: возвращает *wall time* в секундах с момента первого вызова `clock()`. Точность обычно лучше 1 микросекунды.

**Deprecated.**

Важной характеристикой счётчиков является точность. Функция `clock_getres(clk_id)` возвращает точность счётчика с переданным `clk_id`.

In [3]:
possible_clk_id_constants = ['CLOCK_BOOTTIME', 'CLOCK_HIGHRES', 'CLOCK_MONOTONIC', 'CLOCK_MONOTONIC_RAW', 'CLOCK_PROCESS_CPUTIME_ID', 'CLOCK_PROF', \
                    'CLOCK_THREAD_CPUTIME_ID', 'CLOCK_UPTIME', 'CLOCK_REALTIME']
for clk_id_constant in possible_clk_id_constants:
    if hasattr(time, clk_id_constant):
        print(clk_id_constant, ':', time.clock_getres(getattr(time, clk_id_constant)))

CLOCK_MONOTONIC : 1.0000000000000002e-06
CLOCK_MONOTONIC_RAW : 1e-09
CLOCK_PROCESS_CPUTIME_ID : 1.0000000000000002e-06
CLOCK_THREAD_CPUTIME_ID : 1e-09
CLOCK_REALTIME : 1.0000000000000002e-06


Пусть произошло исполнение двух строчек кода 1) `time_0 = timer()`, 2) `time_1 = timer()`. Нумерация отражает порядок исполнения в реальном времени. Счётчики, для которых гарантируется, что `time_0` $\le$`time_1`, называются монотонными.

Если счётчик не является монотонным и между исполнениями строчек 1) и 2) произошёл сдвиг системных часов, `time_0` может быть больше, чем `time_1`. Системные часы могут быть сдвинуты вследствие действий системного администратора или NTP adjustments.

In [4]:
for name in ['clock', 'monotonic', 'perf_counter', 'process_time', 'thread_time', 'time']:
    print(name, ':', time.get_clock_info(name))

clock : namespace(adjustable=False, implementation='clock()', monotonic=True, resolution=1e-06)
monotonic : namespace(adjustable=False, implementation='mach_absolute_time()', monotonic=True, resolution=1e-09)
perf_counter : namespace(adjustable=False, implementation='mach_absolute_time()', monotonic=True, resolution=1e-09)
process_time : namespace(adjustable=False, implementation='clock_gettime(CLOCK_PROCESS_CPUTIME_ID)', monotonic=True, resolution=1.0000000000000002e-06)
thread_time : namespace(adjustable=False, implementation='clock_gettime(CLOCK_THREAD_CPUTIME_ID)', monotonic=True, resolution=1e-09)
time : namespace(adjustable=True, implementation='clock_gettime(CLOCK_REALTIME)', monotonic=False, resolution=1.0000000000000002e-06)


adjustable = True, если счётчик может быть изменён.

In [ ]:
time.clock_gettime(clk_id) → float
time.clock_gettime_ns(clk_id) → int
time.clock_settime(clk_id, time: float)
time.clock_settime_ns(clk_id, time: int)

In [6]:
time.clock_settime(time.CLOCK_THREAD_CPUTIME_ID, 0)

OSError: [Errno 22] Invalid argument

In [7]:
time.clock_settime(time.CLOCK_REALTIME, 0)

PermissionError: [Errno 1] Operation not permitted

In [9]:
time_0 = time.monotonic()
time.sleep(1)
time_1 = time.monotonic()
time_1 - time_0

1.0022855059999927

Возвращает монотонное *wall time* время в секундах с неопределённого момента. Значение не зависит от изменений системных часов.

In [10]:
time_0 = time.perf_counter()
time.sleep(1)
time_1 = time.perf_counter()
time_1 - time_0

1.004135544999997

Возвращает *wall time* время счётчика с наибольшей точностью в секундах с неопределённого момента.

In [11]:
time_0 = time.process_time()
time.sleep(1)
time_1 = time.process_time()
time_1 - time_0

0.0037289999999998713

Возвращает сумму *system time* и *user time* текущего процесса в секундах с неопределённого момента.

In [12]:
time_0 = time.thread_time()
time.sleep(1)
time_1 = time.thread_time()
time_1 - time_0

0.000260637000000008

Аналог `time.process_time()` для тредов.

In [13]:
time.time()

1553291783.246113

Возвращает *wall time* с эпохи в секундах. Обычно эпоха = 01.01.1970 00:00:00 (UTC), также эпоху возвращает `gmtime(0)`.

Если между вызовами системные часы были изменены, то возвращаемые значения могут быть немонотонными.

In [14]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

In [ ]:
time.pthread_getcpuclockid(thread_id) # возвращает clk_id счётчика, который возвращает CPU time для thread c переданным thread_id

## `asyncio.loop.time()`

In [ ]:
# base_event.py

class BaseEventLoop(events.AbstractEventLoop):
  def time(self):
          """Return the time according to the event loop's clock.

          This is a float expressed in seconds since an epoch, but the
          epoch, precision, accuracy and drift are unspecified and may
          differ per event loop.
          """
          return time.monotonic()

In [17]:
import asyncio

loop = asyncio.new_event_loop()

for i in range(5):
  await asyncio.sleep(1)
  print(loop.time())

loop.stop()
loop.close()

190.3393646
191.339844045
192.340572025
193.343911783
194.34912402


## `timeit`

Модуль применяется для бенчмаркинга.

Пример

In [18]:
import timeit
timeit.timeit(stmt='"g" in text', setup='text = "sample string"')

0.05260035500000981

Основные функции

In [ ]:
default_timer = time.perf_counter

timeit(stmt='pass', setup='pass', timer=default_timer, number=1000000, globals=None) 
# вызывает setup 1 раз, вызывает stmt number раз, возвращает среднее время
# перед замером времени выключается сборщик мусора

repeat(stmt='pass', setup='pass', timer=default_timer, repeat=5, number=1000000, globals=None) 
# вызывает timeit repeat раз, возвращает лист времён

T = Timer(stmt="pass", setup="pass", timer=default_timer, globals=None)
T.autorange(callback=None)
# вызывает timeit с возрастающими значениями number пока замеряемое время < 0.2 секунд

Вычисление времени исполнения производится с помощью счётчика `timer`, значение по умолчанию - `perf_counter` из модуля `time`.